In [1]:
import numpy as np
import json
from pathlib import Path

from matrix import saveMatrices

np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)


In [2]:
def genRotX(angle):
    return np.array(
        [
            [1, 0, 0, 0],
            [0, np.cos(angle), -np.sin(angle), 0],
            [0, np.sin(angle), np.cos(angle), 0],
            [0, 0, 0, 1],
        ]
    )


def genRotY(angle):
    return np.array(
        [
            [np.cos(angle), 0, np.sin(angle), 0],
            [0, 1, 0, 0],
            [-np.sin(angle), 0, np.cos(angle), 0],
            [0, 0, 0, 1],
        ]
    )


def genRotZ(angle):
    return np.array(
        [
            [np.cos(angle), -np.sin(angle), 0, 0],
            [np.sin(angle), np.cos(angle), 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
        ]
    )


def genTransX(shift):
    return np.array(
        [
            [1, 0, 0, shift],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
        ]
    )


def genTransY(shift):
    return np.array(
        [
            [1, 0, 0, 0],
            [0, 1, 0, shift],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
        ]
    )


def genTransZ(shift):
    return np.array(
        [
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, shift],
            [0, 0, 0, 1],
        ]
    )


In [3]:
# general misalignment matrix with uniform misalignment in all directions/rotations
def genMatrix(avgShift, avgRot):
    shiftVals = np.random.normal(0, avgShift, 3)
    rotVals = np.random.normal(0, avgRot, 3)

    # matrix multiplication is right->left
    # apply in the order: X -> Y -> Z -> rotX -> rotY -> rotZ
    totalTrans = (
        genTransZ(shiftVals[2]) @ genTransY(shiftVals[1]) @ genTransX(shiftVals[0])
    )
    totalRot = genRotZ(rotVals[2]) @ genRotY(rotVals[1]) @ genRotX(rotVals[0])
    return totalRot @ totalTrans


# sensor matrices are glued to the modulke, so they can't rotate out of the plane of the module
# they can only shift in X,Y and rotate about Z
def genSensorMatrix(avgShift, avgRot):
    shiftVals = np.random.normal(0, avgShift, 2)
    rotVals = np.random.normal(0, avgRot, 1)
    totalTrans = genTransY(shiftVals[1]) @ genTransX(shiftVals[0])
    totalRot = genRotZ(rotVals[0])
    return totalRot @ totalTrans

# module matrices follow the same reasoning as sensors, 
# but we need also the 1/4th of every transformation for the avg misalign (external matrix)
def genModuleMatrix(avgShift, avgRot):
    shiftVals = np.random.normal(0, avgShift, 2)
    rotVals = np.random.normal(0, avgRot, 1)
    totalTrans = genTransY(shiftVals[1]) @ genTransX(shiftVals[0])
    totalTransQ = genTransY(shiftVals[1]/4) @ genTransX(shiftVals[0]/4)
    totalRot = genRotZ(rotVals[0])
    totalRotQ = genRotZ(rotVals[0]/4)
    return totalRot @ totalTrans, totalRotQ @ totalTransQ


In [4]:
# generate sector-> modules in this sector lookup
with open('input/moduleIDtoSectorID.json') as f:
    modToSec = json.load(f)

with open('input/moduleIDtoModulePath.json') as f:
    modToPath = json.load(f)

sectors = []
for keys, values in modToSec.items():
    sectors.append(values)

sectors = sorted(set(sectors))
sectorPaths = {}

for sector in sectors:
    sectorPaths[str(sector)] = []
    for key, value in modToSec.items():
        if value == sector:
            sectorPaths[str(sector)].append(modToPath[key]) 

with open('input/sectorPaths.json', 'w') as f:
    json.dump(sectorPaths, f, sort_keys=True, indent=2)

In [6]:
# make this repeatable
np.random.seed(12345)

# open available paths
with open("input/assemblyPaths.json") as f:
    allPaths = json.load(f)

# remember, ROOT units are cm
# BUT rotation valus must still be radian

# avg values for box, 100mu and 1mrad
avgShiftInCM = 100e-4
avgRotInDeg = 1e-3

misAlignBox = {}
for pathBox in allPaths["box"]:
    misAlignBox[pathBox] = genMatrix(avgShiftInCM, avgRotInDeg)
saveMatrices(misAlignBox, "output/misMat-box.json")

# avg vales for sensors
avgShiftInCM = 100e-4
avgRotInDeg = avgShiftInCM / 4

misalignModules = {}
misalignModulesQ = {}
externalMatricesModules = {}
for pathBox in allPaths["modules"]:
    misalignModules[pathBox], misalignModulesQ[pathBox] = genModuleMatrix(avgShiftInCM, avgRotInDeg)

# gen external matrices
for key, value in sectorPaths.items():
    thisExternalMatrix = np.eye(4)
    for path in value:
        thisExternalMatrix = thisExternalMatrix @ misalignModulesQ[path]
    externalMatricesModules[key] = thisExternalMatrix
    

saveMatrices(misalignModules, "output/misMat-modules.json")
saveMatrices(externalMatricesModules, "output/externalMatrices-modules.json")

# avg vales for modules
avgShiftInCM = 100e-4
avgRotInDeg = 0.014

misalignSensors = {}
externalMatricesSensors = {}
for pathBox in allPaths["sensors"]:
    misalignSensors[pathBox] = genSensorMatrix(avgShiftInCM, avgRotInDeg)
    if pathBox.endswith('sensor_0') or pathBox.endswith('sensor_1'):
        externalMatricesSensors[pathBox] = misalignSensors[pathBox]

saveMatrices(misalignSensors, "output/misMat-sensors.json")
saveMatrices(externalMatricesSensors, "output/externalMatrices-sensors.json")



WARNING. Replacing file: output/misMat-box.json!

WARNING. Replacing file: output/misMat-modules.json!

WARNING. Replacing file: output/externalMatrices-modules.json!

WARNING. Replacing file: output/misMat-sensors.json!

WARNING. Replacing file: output/externalMatrices-sensors.json!

